# **Lab1: Regression**
In *lab 1*, you need to finish:

1.  Basic Part: Implement the regression model to predict people's grip force from their weight.
You can use either Matrix Inversion or Gradient Descent.


> *   Step 1: Split Data
> *   Step 2: Preprocess Data
> *   Step 3: Implement Regression
> *   Step 4: Make Prediction
> *   Step 5: Train Model and Generate Result

2.  Advanced Part: Implementing a regression model to predict grip force in a different way (for example, with more variables) than the basic part




---
# 1. Basic Part (50%)
In the first part, you need to implement the regression to predict grip force

Please save the prediction result in a CSV file and submit it to Kaggle

### Import Packages

> Note: You **cannot** import any other package


In [365]:
#pip install numpy matplotlib pandas

In [366]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import csv
import math
import random

### Global attributes
Define the global attributes\
You can also add your own global attributes here

In [367]:
training_dataroot = 'lab1_basic_training.csv' # Training data file file named as 'lab1_basic_training.csv'
testing_dataroot = 'lab1_basic_testing.csv'   # Testing data file named as 'lab1_basic_testing.csv'
output_dataroot = 'lab1_basic.csv' # Output file will be named as 'lab1_basic.csv'

training_datalist =  [] # Training datalist, saved as numpy array
testing_datalist =  [] # Testing datalist, saved as numpy array

output_datalist =  [] # Your prediction, should be a list with 100 elements

### Load the Input File
First, load the basic input file **lab1_basic_training.csv** and **lab1_basic_testing.csv**

Input data would be stored in *training_datalist* and *testing_datalist*

In [368]:
# Read input csv to datalist
with open(training_dataroot, newline='') as csvfile:
  training_datalist = pd.read_csv(training_dataroot).to_numpy()

with open(testing_dataroot, newline='') as csvfile:
  testing_datalist = pd.read_csv(testing_dataroot).to_numpy()

### Implement the Regression Model

> Note: It is recommended to use the functions we defined, you can also define your own functions

#### Step 1: Split Data
Split data in *training_datalist* into training dataset and validation dataset


In [369]:
def SplitData(data, split_ratio):
    """
    Splits the given dataset into training and validation sets based on the specified split ratio.

    Parameters:
    - data (numpy.ndarray): The dataset to be split. It is expected to be a 2D array where each row represents a data point and each column represents a feature.
    - split_ratio (float): The ratio of the data to be used for training. For example, a value of 0.8 means 80% of the data will be used for training and the remaining 20% for validation.

    Returns:
    - training_data (numpy.ndarray): The portion of the dataset used for training.
    - validation_data (numpy.ndarray): The portion of the dataset used for validation.

    """
    data = np.array(data) # transfer data into numpy array
    np.random.shuffle(data) # shuffle data
    split_index = int(len(data) * split_ratio) # calculate data length to split
    training_data = data[:split_index] # Put data[0] to data[split_index-1] into training_data
    validation_data = data[split_index:] # Put data[split_index] to the last element to validation_data

    # TODO

    return training_data, validation_data



#### Step 2: Preprocess Data
Handle unreasonable data and missing data

> Hint 1: Outliers and missing data can be addressed by either removing them or replacing them using statistical methods (e.g., the mean of all data).

> Hint 2: Missing data are represented as `np.nan`, so functions like `np.isnan()` can be used to detect them.

> Hint 3: Methods such as the Interquartile Range (IQR) can help detect outliers

In [370]:
def PreprocessData(data):
    """
    Preprocess the given dataset and return the result.

    Parameters:
    - data (numpy.ndarray): The dataset to preprocess. It is expected to be a 2D array where each row represents a data point and each column represents a feature.

    Returns:
    - preprocessedData (numpy.ndarray): Preprocessed data.
    """
    data = np.array(data) # transfer data into numpy array
    # y=np.array(data[:,1])
    # nan_mask = np.isnan(y)
    # cleaned_data = data[~nan_mask]

    preprocessedData1 = [ row for row in data if not np.isnan(row[1])] # put data into preprocessedData if is not a missing data
    # replace missing value with mean value
    preprocessedData2 = data.copy() # copy data to data2
    mean = np.nanmean(preprocessedData2[:, 1])
    for i in range(len(preprocessedData2)):
        if not np.isnan(preprocessedData2[i][1]):continue # skip not missing data
        preprocessedData2[i][1] = mean # replace missing data with mean value

    # replace missing value with IQR
    preprocessedData3 = data.copy()
    sorted_data = preprocessedData3[preprocessedData3[:, 1].argsort()]
    sorted_data_2 = sorted_data[:, 1]
    Q1 = np.percentile(sorted_data_2, 25) # set Q1 is the no. 25%
    Q3 = np.percentile(sorted_data_2, 75) # set Q3 is the no. 75%
    IQR = Q3 - Q1
    for i in range(len(preprocessedData3)):
        if not np.isnan(preprocessedData3[i][1]):continue # skip not missing data
        preprocessedData3[i][1] = IQR # replace missing data with IQR

    # TODO

    preprocessedData1 = np.array(preprocessedData1)
    preprocessedData2 = np.array(preprocessedData2)
    preprocessedData3 = np.array(preprocessedData3)
    return preprocessedData3#,preprocessedData2,preprocessedData3


### Step 3: Implement Regression
You have to use Gradient Descent to finish this part

In [371]:
def Regression(dataset):
    """
    Performs regression on the given dataset and return the coefficients.

    Parameters:
    - dataset (numpy.ndarray): A 2D array where each row represents a data point.

    Returns:
    - w (numpy.ndarray): The coefficients of the regression model. For example, y = w[0] + w[1] * x + w[2] * x^2 + ...
    """
    #print(type(dataset))
    X = dataset[:, :1]
    y = dataset[:, 1]

    # TODO: Decide on the degree of the polynomial
    degree = 2  # For example, quadratic regression, phi^i(x) = x^i

    
    # Add polynomial features to X
    X_poly = np.ones((X.shape[0], 1))  # Add intercept term (column of ones)
    for d in range(1, degree + 1):
        X_poly = np.hstack((X_poly, X ** d))  # Add x^d terms to feature matrix

    X_log = []# All x_i = log(x_i + 1),phi(x) = log(x+1)
    for i in range(len(y)):
        X_log.append(math.log(X[i] + 1))
    X_log=np.array(X_log)
    #print(X_poly.shape)
    # Initialize coefficients (weights) to zero
    num_dimensions = X_poly.shape[1]  # Number of features (including intercept and polynomial terms)
    w = np.zeros(num_dimensions) #initial w
    w2 = np.zeros(1)
    w2 = np.array(w2)

    # TODO: Set hyperparameters
    num_iteration = 1000
    learning_rate = 0.01


    # Functions definitions (phi) 
    #phi = [X_poly,X_log]# len(phi) need to be greater than number of features

    #num_features = X.shape[1]

    # Gradient Descent
    m = len(y)  # Number of data points
    #print(X_poly[5000][2])
    for iteration in range(num_iteration):
        
        # TODO: Prediction using current weights and compute error
        #y_prediction = np.zeros(m)
        #w_features = []
        #for i in range(m):
        #y_prediction = X_poly.dot(w)
            # for j in range(num_dimensions):
            #     #w_features.append(w)
            #     #w_features=np.array(w_features)
            #     y_prediction[i] += w[j][i]*X_poly[j][i]

        #predict with log(x+1)
        
        y_log_pred = np.array(X_log * w2)
        #print(f"y_log_pred : {y_log_pred}")
        # TODO: Compute gradient
        grad=[]
        #error = y-y_prediction
        different2 = y-y_log_pred
        #print(f"error2 : {error2}")
        gradient2 = np.nansum(np.array(different2*X_log)) * -2/m
        # for i in range(num_dimensions):
        #     for j in range(m):
        #         g = error2[j] * X_poly[j][i]
        #     gradient=np.sum(g,axis=0)
        #     grad.append(-2/m *gradient)
        #     #print(f"grad : {grad}, gradient : {gradient}")
        # grad = np.array(grad)

        # print(f"grad : {grad}, gradient : {gradient}")
        # error = y-y_prediction
        # gradient = -1 * 2/m * X_poly.T.dot(error)

        #Fail cause tmp_sumation get nan
        # tmp_sumation = 0
        # for i in range(m):
        #     tmp_sumation+=error2[i] * X_log[i]
        # print(f"tmp_sum : {tmp_sumation}")
        # gradient3 = tmp_sumation * -2/m

        # TODO: Update the weights
        #w -= grad*learning_rate
        old_w = w2
        w2 -= gradient2*learning_rate
        if abs(old_w - w2) < 0.000001: break
        np.set_printoptions(precision=2, suppress=True)
        #print(f"w: {w}, grad : {grad}, error : {error}")
        

        # TODO: Optionally, print the cost every 100 iterations
        if iteration % 100 == 0:
            cost = np.nansum((y-y_log_pred)**2)/m # set MSE as cost funciton
            print(f"Iteration {iteration}, Cost: {cost}, w: {w2}, grad2 : {gradient2}")

    return w2


### Step 4: Make Prediction
Make prediction of testing dataset and store the value in *output_datalist*

In [372]:
def MakePrediction(w, test_dataset):
    """
    Predicts the output for a given test dataset using a regression model.

    Parameters:
    - w (numpy.ndarray): The coefficients of the model, where each element corresponds to
                               a coefficient for the respective power of the independent variable.
    - test_dataset (numpy.ndarray): A 1D array containing the input values (independent variable)
                                          for which predictions are to be made.

    Returns:
    - list/numpy.ndarray: A list or 1d array of predicted values corresponding to each input value in the test dataset.
    """
    w=np.array(w)
    test_dataset = np.array(test_dataset)
    prediction = test_dataset.dot(w)
    # X = test_dataset[:, :1]
    # y = test_dataset[:, 1]
    # prediction = []

    # TODO

    return prediction


### Step 5: Train Model and Generate Result

Use the above functions to train your model on training dataset, and predict the answer of testing dataset.

Save your predicted values in `output_datalist`

> Notice: **Remember to inclue the coefficients of your model in the report**



In [373]:
# TODO
def BasicRegression(dataset):
    X = dataset[:, :1]
    y = dataset[:, 1]
    tmp = []
    tmp2 = []
    for i in range(len(y)):
        if not np.isnan(y[i]) and not np.isnan(X[i]) :
            tmp.append(X[i] * y[i])
            tmp2.append(X[i] * X[i])
    Sum_XY = 0
    Sum_X2 =0
    for i in range(len(tmp)):
        Sum_XY += tmp[i]
        Sum_X2 += tmp2[i]
    print(f" {Sum_X2} : {Sum_XY}")


    return Sum_XY/Sum_X2
# (1) Split data
TraningData,ValidationData = SplitData(training_datalist,0.8)

# (2) Preprocess data
ProcessedData1 = PreprocessData(TraningData)
ProcessedData2 = PreprocessData(ValidationData)
#print(ProcessedData1)
#print(ProcessedData1)
# (3) Train regression model
w=Regression(ProcessedData1)
output_datalist = MakePrediction(w,testing_datalist)
print(f"Result : {output_datalist}")

# (4) Predict validation dataset's answer, calculate MAPE comparing to the ground truth

# (5) Make prediction of testing dataset and store the values in output_datalist


Result : [221.11 212.61 328.77 233.81 282.57 267.31 236.78 261.94 261.12 253.28
 288.34 381.57 342.8  211.62 266.07 264.83 267.31 236.37 228.94 278.03
 346.51 273.91 349.81 339.5  212.03 377.03 320.52 232.66 273.49 217.81
 341.97 284.22 264.83 264.42 313.92 238.1  186.54 373.73 181.92 251.22
 207.49 284.63 329.18 245.86 212.44 324.65 232.66 221.11 214.92 273.91
 310.21 231.01 247.51 285.04 255.76 303.19 246.68 326.3  218.22 350.63
 340.73 201.3  348.98 331.66 303.19 357.65 238.43 202.95 348.16 270.19
 370.85 276.22 351.05 328.77 432.72 223.91 233.07 282.57 317.3  400.13
 276.79 299.07 261.12 263.59 300.31 269.37 212.85 257.41 216.16 240.91
 198.83 309.38 230.59 228.12 322.99 229.77 358.47 290.82 344.45 216.57]


/var/folders/g4/6y44v3pd2yz970m11925pw_40000gn/T/ipykernel_24166/648953006.py:26: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  X_log.append(math.log(X[i] + 1))


### *Write the Output File*

Write the prediction to output csv and upload the file to Kaggle
> Format: 'Id', 'gripForce'


In [374]:
# Assume that output_datalist is a list (or 1d array) with length = 100
with open(output_dataroot, 'w', newline='', encoding="utf-8") as csvfile:
  writer = csv.writer(csvfile)
  writer.writerow(['Id', 'gripForce'])
  for i in range(len(output_datalist)):
    writer.writerow([i,output_datalist[i]])


# 2. Advanced Part (45%)
In the second part, you need to implement regression differently from the basic part to improve your grip force predictions. You must use more than two features.

You can choose either matrix inversion or gradient descent for this part

We have provided `lab1_advanced_training.csv` for your training

> Notice: Be cautious of the "gender" attribute, as it is represented by "F"/"M" rather than a numerical value.

Please save the prediction result in a CSV file and submit it to Kaggle

In [375]:
training_dataroot = 'lab1_advanced_training.csv' # Training data file file named as 'lab1_advanced_training.csv'
testing_dataroot = 'lab1_advanced_testing.csv'   # Testing data file named as 'lab1_advanced_testing.csv'
output_dataroot = 'lab1_advanced.csv' # Output file will be named as 'lab1_advanced.csv'

training_datalist =  [] # Training datalist, saved as numpy array
testing_datalist =  [] # Testing datalist, saved as numpy array

output_datalist =  [] # Your prediction, should be a list with 3000 elements

In [376]:
# Read input csv to datalist
with open(training_dataroot, newline='') as csvfile:
  training_datalist = pd.read_csv(training_dataroot).to_numpy()

with open(testing_dataroot, newline='') as csvfile:
  testing_datalist = pd.read_csv(testing_dataroot).to_numpy()

In [377]:
# TODO

# (1) Split data

# (2) Preprocess data

# (3) Train regression model

# (4) Predict validation dataset's answer, calculate MAPE comparing to the ground truth

# (5) Make prediction of testing dataset and store the values in output_datalist

# Save the Code File
Please save your code and submit it as an ipynb file! (**Lab1.ipynb**)